In [ ]:
#############
#todo
#############
#CLIP
crop

#reproject

spTransform

#sampling z bodu
    #nahrát body    
    #nasamplovat hodnoty k bodum

s<-extract(st_prox,point,sp=T)


#zonal statistika
    #polygony prumery


#maskovani/mraky


